In [3]:
from WindPy import w
w.start()

.ErrorCode=0
.Data=[Already connected!]

In [1]:
from Modules import PortfolioState
from Modules import DateHelper
from dateutil.relativedelta import relativedelta


start_date = '20240126'
date_helper = DateHelper()

In [3]:
from dataclasses import dataclass, field


@dataclass
class WeeklyData:
    signal_for_next_week = None
    signal_in_week = None
    
    repositioning_day_first = None
    repositioning_day_first_close = None
    repositioning_day_first_stage_one_money = None
    repositioning_day_first_stage_two_money = None
    
    repositioning_day_second = None
    repositioning_day_second_close = None
    repositioning_day_second_stage_one_money = None
    repositioning_day_second_stage_two_money = None

    def show(self) -> None:
        """
        打印出本周数据的当前状态，包括所有属性和它们的值。
        """
        for attr, value in self.__dict__.items():
            print(f"{attr}: {value}")

In [2]:
import datetime

class Week:
    def __init__(self, helper, mark_day) -> None:
        self.start, self.end = helper.get_target_week(mark_day)
        self.trading_days = w.tdays(self.start, self.end, '').Data[0]
        self.first_repositioning_day = self.trading_days[0]
        self.final_signal_day = self.trading_days[-1]
        self.set_intra_week_signal_day()
        
    @staticmethod
    def get_weekday_name(date_obj):
        weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
        return weekdays[date_obj.weekday()]
    
    def is_first_repositioning_day_after_tuesday(self):
        weekday_index = self.first_repositioning_day.weekday()
        return weekday_index > 1
    
    def set_intra_week_signal_day(self):
        # 检查是否有周二
        tuesday = next((day for day in self.trading_days if day.weekday() == 1), None)
        # 如果没有周二，检查是否有周一
        monday = next((day for day in self.trading_days if day.weekday() == 0), None)
        
        if tuesday:
            self.intra_week_signal_day = tuesday
        elif monday:
            self.intra_week_signal_day = monday
        else:
            self.intra_week_signal_day = None


week = Week(date_helper, '20240501')
week.trading_days

NameError: name 'date_helper' is not defined

In [1]:
import datetime
from typing import Union
from Modules import DateHelper, StockDataFetcher
import pandas as pd
from typing import Tuple, List
import WindPy


class TradingDay:
    """
    一个封装了 datetime.datetime 功能的类，支持初始化时传入字符串或 datetime.datetime 对象。

    Attributes:
        dt (datetime.datetime): 内部的 datetime.datetime 实例。
    """

    def __init__(self,
                 input_date: Union[str, datetime.datetime],
                 date_helper: DateHelper = DateHelper(),
                 stock_helper: StockDataFetcher = StockDataFetcher(),
                 ) -> None:
        """
        初始化 TradingDay 类。

        参数:
        input_date (Union[str, datetime.datetime]): 输入的日期，可以是字符串或 datetime.datetime 对象。
        """
        self.date_helper = date_helper
        self.stock_helper = stock_helper
        if isinstance(input_date, str):
            self.dt = self.date_helper.convert_to_timestamp(input_date)
        elif isinstance(input_date, datetime.datetime):
            self.dt = input_date
        else:
            raise ValueError("input_date 必须是字符串或 datetime.datetime 类型。")

        if not self.date_helper.is_trading_day(self.dt):
            raise ValueError("给定日期不是交易日。")

    def close(self, code: str, close_type: str = "close", priceAdj: str = "U", cycle: str = "D"):
        return self.stock_helper.close(code=code, close_type=close_type, priceAdj=priceAdj, cycle=cycle)

    def minute_price(self,
                     code: str,
                     start_date: Union[str, datetime.datetime] = None,
                     end_date: Union[str, datetime.datetime] = None,
                     frequency: str = '1'
                     ) -> pd.DataFrame:
        """
        获取股票的收盘价数据。

        参数:
        code (str): 股票代码。
        start_date (str or datetime.datetime, optional): 开始日期, 默认为昨天。
            - 如果是 str, 支持 "YYYYMMDD", "YYYY-MM-DD", "YYYYMMDD HH:MM:SS", "YYYY-MM-DD HH:MM:SS" 格式。
            - 如果是 datetime.datetime, 将直接使用该对象。
            默认值: None, 表示使用昨天的日期。
        end_date (str or datetime.datetime, optional): 结束日期, 默认为当前时间。
            - 如果是 str, 支持 "YYYYMMDD", "YYYY-MM-DD", "YYYYMMDD HH:MM:SS", "YYYY-MM-DD HH:MM:SS" 格式。
            - 如果是 datetime.datetime, 将直接使用该对象。
            默认值: None, 表示使用当前时间。
        frequency (str, optional): 数据获取频率, 默认为 '1' 分钟。
            - 可以是 '1', '3', '5', '10', '15', '30', '60' 等分钟级别, 或者 'D' 日频, 'W' 周频等。

        返回:
        pandas.DataFrame: 包含收盘价数据的 DataFrame。

        示例:
        >>> DataFetcher = DataFetcher()
        >>> data_df = DataFetcher.get_stock_close_price('000852.SH')
        >>> print(data_df)
        """

        return self.stock_helper.get_stock_price(code=code, start_date=start_date, end_date=end_date,
                                                 frequency=frequency)

    def __getattr__(self, name):
        """
        代理所有未定义的属性和方法到内部的 datetime.datetime 实例。
        """
        return getattr(self.dt, name)

    def __repr__(self):
        """
        返回对象的字符串表示形式。
        """
        return repr(self.dt)

    def __str__(self):
        """
        返回对象的字符串表示形式。
        """
        return str(self.dt)

    def format(self, fmt: str) -> str:
        """
        格式化日期时间字符串。

        参数:
        fmt (str): 格式字符串，如 '%Y-%m-%d %H:%M:%S'。

        返回:
        str: 格式化的日期时间字符串。
        """
        return self.dt.strftime(fmt)

    def __add__(self, other: datetime.timedelta) -> 'TradingDay':
        """
        支持加法操作。

        参数:
        other (timedelta): 要加上的时间间隔。

        返回:
        TradingDay: 新的 TradingDay 实例。
        """
        new_dt = self.dt + other
        return TradingDay(new_dt)

    def __sub__(self, other: Union['TradingDay', datetime.timedelta]) -> Union['TradingDay', datetime.timedelta]:
        """
        支持减法操作。

        参数:
        other (Union[TradingDay, timedelta]): 要减去的时间间隔或 TradingDay 实例。

        返回:
        Union[TradingDay, timedelta]: 新的 TradingDay 实例或时间间隔。
        """
        if isinstance(other, datetime.timedelta):
            new_dt = self.dt - other
            return TradingDay(new_dt)
        elif isinstance(other, TradingDay):
            return self.dt - other.dt
        else:
            raise TypeError("Unsupported operand type for -: 'TradingDay' and '{}'".format(type(other).__name__))

    def __lt__(self, other):
        if isinstance(other, TradingDay):
            return self.dt < other.dt
        elif isinstance(other, datetime.datetime):
            return self.dt < other
        else:
            raise TypeError("Unsupported operand type for <: 'TradingDay' and '{}'".format(type(other).__name__))

    def __le__(self, other):
        if isinstance(other, TradingDay):
            return self.dt <= other.dt
        elif isinstance(other, datetime.datetime):
            return self.dt <= other
        else:
            raise TypeError("Unsupported operand type for <=: 'TradingDay' and '{}'".format(type(other).__name__))

    def __gt__(self, other):
        if isinstance(other, TradingDay):
            return self.dt > other.dt
        elif isinstance(other, datetime.datetime):
            return self.dt > other
        else:
            raise TypeError("Unsupported operand type for >: 'TradingDay' and '{}'".format(type(other).__name__))

    def __ge__(self, other):
        if isinstance(other, TradingDay):
            return self.dt >= other.dt
        elif isinstance(other, datetime.datetime):
            return self.dt >= other
        else:
            raise TypeError("Unsupported operand type for >=: 'TradingDay' and '{}'".format(type(other).__name__))
    

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.
Wind terminal started successfully.


In [3]:
a = TradingDay('20240426')
a - datetime.timedelta(days=1)

Timestamp('2024-04-25 00:00:00')

In [4]:
from WindPy import w

In [5]:
import datetime
import WindPy


class TradingWeek:
    def __init__(self, mark_day: Union[str, datetime.datetime, TradingDay], w: WindPy.w, helper: DateHelper = DateHelper(), intra_reposition: bool = True) -> None:
        """
        初始化 Week 类，用于计算指定周的交易日信息。

        参数:
        helper (DateHelper): 提供辅助方法的对象，例如获取目标周。
        mark_day (Union[str, datetime.datetime, TradingDay]): 用于计算目标周的标记日期。
        w (WindPy.w): Wind API, 用于获取交易日。
        intra_reposition (bool): 是否启用周内重新定位, 默认为True。
        """
        self.w = w
        self.intra_reposition = intra_reposition
        if isinstance(mark_day, TradingDay):
            mark_day = mark_day.dt
        self.start, self.end = helper.get_target_week(mark_day)
        self.trading_days = self._get_trading_days()
        self.early_week_days, self.late_week_days = self._split_week_days()

        if not self.trading_days:
            self.has_trading_days = False
            self.no_trading_days_error()
        else:
            self.has_trading_days = True
            if isinstance(self.trading_days[0], datetime.datetime):
                self.first_repositioning_day = TradingDay(self.trading_days[0])
            else:
                self.first_repositioning_day = self.trading_days[0]
            if isinstance(self.trading_days[-1], datetime.datetime):
                self.final_signal_generating_day = TradingDay(self.trading_days[-1])
            else:
                self.final_signal_generating_day = self.trading_days[-1]

            if not self.early_week_days:
                self.intra_reposition = False

            if self.intra_reposition:
                if isinstance(self.early_week_days[-1], datetime.datetime):
                    self.intra_signal_generating_day = TradingDay(self.early_week_days[-1])
                else:
                    self.intra_signal_generating_day = self.early_week_days[-1]
                if self.late_week_days and isinstance(self.late_week_days[0], datetime.datetime):
                    self.intra_repositioning_day = TradingDay(self.late_week_days[0])
                else:
                    self.intra_repositioning_day = self.late_week_days[0] if self.late_week_days else self.early_week_days[-1]

    def _get_trading_days(self) -> List[TradingDay]:
        """
        使用 WindPy 库获取从开始日期到结束日期之间的所有交易日。
        """
        trading_days_datetime = self.w.tdays(self.start, self.end, '').Data[0]
        trading_days = [TradingDay(day) if isinstance(day, datetime.datetime) else day for day in trading_days_datetime]
        return trading_days

    def _split_week_days(self) -> Tuple[List[TradingDay], List[TradingDay]]:
        """
        将交易日分为两个列表：早于等于周二的交易日和晚于周二的交易日。
        如果找不到周二的交易日, 则所有交易日都视为早期交易日。
        """
        tuesday = next((day for day in self.trading_days if day.weekday() == 1), None)
        if tuesday:
            early_week = [day for day in self.trading_days if day <= tuesday]
            late_week = [day for day in self.trading_days if day > tuesday]
        else:
            early_week = list(self.trading_days)
            late_week = []
        return early_week, late_week

    def no_trading_days_error(self) -> None:
        """
        当没有交易日时触发的错误处理方法。
        抛出 ValueError 异常。
        """
        raise ValueError("在开始和结束日期之间未找到交易日。")
    
    

In [6]:
q = TradingWeek('20240501', w)

In [7]:
q.early_week_days[0].close('000852.SH')

.ErrorCode=0
.Codes=[000852.SH]
.Fields=[CLOSE]
.Times=[20240902 13:10:51]
.Data=[[4629.3195]]

In [46]:
w.wss("000001.SZ", "close,open,high,low","tradeDate=20240821;priceAdj=U;cycle=D")

.ErrorCode=0
.Codes=[000001.SZ]
.Fields=[CLOSE,OPEN,HIGH,LOW]
.Times=[20240902 10:37:45]
.Data=[[10.35],[10.3],[10.38],[10.21]]

In [57]:
a.close('000852.SH')

20240426


.ErrorCode=0
.Codes=[000852.SH]
.Fields=[CLOSE]
.Times=[20240902 10:38:42]
.Data=[[5415.9379]]

In [9]:
aa = w.wss("000001.SZ", "pre_close,pre_close_exch,open,high,low,close,dq_stockclose,volume,volume_btin,amt,amount_btin,dealnum,volume_aht,amount_aht,chg,pct_chg,pct_chg_b,swing,vwap,adjfactor,close2,turn,free_turn_n,dq_amtturnover,oi,oi_chg,oi_index,oichange,oiamount_nomargin,oiamount,pre_settle,settle,dq_settle,chg_settlement,pctchange_close,pct_chg_settlement,lastradeday_s,firstradeday_s,last_trade_day,rel_ipo_chg,rel_ipo_pct_chg,trade_status,val_mv_ARD,val_mvc,susp_days,susp_reason,maxupordown,maxup,maxdown,discount,discount_ratio,open3,high3,low3,close3,indexweight,settle3,oi3,close_FX,close_usd,premiumrate_ah,direction_gold,dquantity_gold,close_night,dq_close_night,free_turn,open_auction_price,open_auction_volume,open_auction_amount","tradeDate=20240901;priceAdj=U;cycle=D;unit=1;bondPriceType=2;adjDate=20240901;windCode3=0")

In [10]:
a

Timestamp('2024-04-26 00:00:00')

In [11]:
a + datetime.timedelta(hours=10) + datetime.timedelta(minutes=30)

Timestamp('2024-04-26 10:30:00')

In [22]:
aa.Data[aa.Fields.index('ADJFACTOR')]

[125.049335]